# Venue Data Analysis of London

### Introduction

In this project, I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in London and their emerging clusters.

### Data

On top of foursquare API (https://developer.foursquare.com/), the following data will be used for this analysis.


Average House Price of London Borough
https://data.london.gov.uk/dataset/average-house-prices

Cultural Infrastructure Map of London
https://data.london.gov.uk/dataset/cultural-infrastructure-map

Coordinates of London boroughs
https://en.wikipedia.org/wiki/List_of_London_boroughs

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [5]:
!pip install folium
import folium

print('Folium imported.')

     |████████████████████████████████| 102kB 8.9MB/s ta 0:00:011
Folium imported.


In [7]:
import os
from bs4 import BeautifulSoup

os.system('wget -q -O "boroughs.html" "https://en.wikipedia.org/wiki/List_of_London_boroughs"')

with open('boroughs.html', encoding='utf-8-sig') as fp:
    soup = BeautifulSoup(fp,"lxml")

data = []
table = soup.find("table", { "class" : "wikitable sortable" })
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append([col.replace(u'\ufeff', '') for col in cols])
print(data)

[[], ['Barking and Dagenham [note 1]', '', '', 'Barking and Dagenham London Borough Council', 'Labour', 'Town Hall, 1 Town Square', '13.93', '194,352', '51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51.5607; 0.1557 (Barking and Dagenham)', '25'], ['Barnet', '', '', 'Barnet London Borough Council', 'Conservative', 'Barnet House, 2 Bristol Avenue, Colindale', '33.49', '369,088', '51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51.6252; -0.1517 (Barnet)', '31'], ['Bexley', '', '', 'Bexley London Borough Council', 'Conservative', 'Civic Offices, 2 Watling Street', '23.38', '236,687', '51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51.4549; 0.1505 (Bexley)', '23'], ['Brent', '', '', 'Brent London Borough Council', 'Labour', 'Brent Civic Centre, Engineers Way', '16.70', '317,264', '51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51.5588; -0.2817 (Brent)', '12'], ['Bromley', '', '', 'Bromley London Borough Council', 'Conservative', 'Civic Centre, Stockwell Close', '57.97', '317,899', '51°24′14″N 0°01′11″E

In [8]:
df = pd.DataFrame(data, columns=['Borough','Inner','Status','Local Authority','Political Control','Headquarters','Area','Population','Co-ordinates','Nr in Maps'])
df.head()

,Borough,Inner,Status,Local Authority,Political Control,Headquarters,Area,Population,Co-ordinates,Nr in Maps
0,None,None,None,None,None,None,None,None,None,None
1,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...,25
2,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...,31
3,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...,23
4,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...,12


In [9]:
df1 = df[['Borough', 'Co-ordinates']].reset_index(drop=True)

In [10]:
df1.head(10)

,Borough,Co-ordinates
0,None,None
1,Barking and Dagenham [note 1],51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...
5,Bromley,51°24′14″N 0°01′11″E / 51.4039°N 0.0198°E / 51...
6,Camden,51°31′44″N 0°07′32″W / 51.5290°N 0.1255°W / 51...
7,Croydon,51°22′17″N 0°05′52″W / 51.3714°N 0.0977°W / 51...
8,Ealing,51°30′47″N 0°18′32″W / 51.5130°N 0.3089°W / 51...
9,Enfield,51°39′14″N 0°04′48″W / 51.6538°N 0.0799°W / 51...


In [11]:
df1["Borough"] = df1["Borough"].str.replace(r"(\s*\[.*?\]\s*)", " ").str.strip()
df1.head(5)

,Borough,Co-ordinates
0,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...


In [12]:
df1.head(5)

,Borough,Co-ordinates
0,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...


In [13]:
df1["Co-ordinates"] = df1["Co-ordinates"].str.replace(r"(\s*\[.*?\]\s*)", " ").str.strip()
df1.head()

,Borough,Co-ordinates
0,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...


In [14]:
df1["Coord"]=df1["Co-ordinates"].str.split("/").str[2]
df1.head()

,Borough,Co-ordinates,Coord
0,None,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...,51.5607; 0.1557 (Barking and Dagenham)
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...,51.6252; -0.1517 (Barnet)
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...,51.4549; 0.1505 (Bexley)
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...,51.5588; -0.2817 (Brent)


In [15]:
df1["Lat"] = df1["Coord"].str.split(";").str[0]
df1.head()

,Borough,Co-ordinates,Coord,Lat
0,None,None,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...,51.5607; 0.1557 (Barking and Dagenham),51.5607
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...,51.6252; -0.1517 (Barnet),51.6252
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...,51.4549; 0.1505 (Bexley),51.4549
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...,51.5588; -0.2817 (Brent),51.5588


In [16]:
df1["Longitude"] = df1["Coord"].str.split(";").str[1]
df1.head()

,Borough,Co-ordinates,Coord,Lat,Longitude
0,None,None,None,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...,51.5607; 0.1557 (Barking and Dagenham),51.5607,0.1557 (Barking and Dagenham)
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...,51.6252; -0.1517 (Barnet),51.6252,-0.1517 (Barnet)
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...,51.4549; 0.1505 (Bexley),51.4549,0.1505 (Bexley)
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...,51.5588; -0.2817 (Brent),51.5588,-0.2817 (Brent)


In [17]:
df1["Long"] = df1["Longitude"].str.split(" ").str[1]
df1.head()

,Borough,Co-ordinates,Coord,Lat,Longitude,Long
0,None,None,None,None,None,None
1,Barking and Dagenham,51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...,51.5607; 0.1557 (Barking and Dagenham),51.5607,0.1557 (Barking and Dagenham),0.1557
2,Barnet,51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...,51.6252; -0.1517 (Barnet),51.6252,-0.1517 (Barnet),-0.1517
3,Bexley,51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...,51.4549; 0.1505 (Bexley),51.4549,0.1505 (Bexley),0.1505
4,Brent,51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...,51.5588; -0.2817 (Brent),51.5588,-0.2817 (Brent),-0.2817


In [18]:
df1.drop('Co-ordinates', axis=1, inplace=True)

In [19]:
df1.drop('Coord', axis=1, inplace=True)
df1.drop('Longitude', axis=1, inplace=True)

In [20]:
df1.head()

,Borough,Lat,Long
0,None,None,None
1,Barking and Dagenham,51.5607,0.1557
2,Barnet,51.6252,-0.1517
3,Bexley,51.4549,0.1505
4,Brent,51.5588,-0.2817


In [21]:
df1.loc[0] = ['City of London','51.5155','-0.0922']
df1.head()

,Borough,Lat,Long
0,City of London,51.5155,-0.0922
1,Barking and Dagenham,51.5607,0.1557
2,Barnet,51.6252,-0.1517
3,Bexley,51.4549,0.1505
4,Brent,51.5588,-0.2817


In [22]:
df1.dtypes

Borough    object
Lat        object
Long       object
dtype: object

In [23]:
df1['Long']=df1['Long'].astype(float)
df1['Lat']=df1['Lat'].astype(float)
df1.dtypes

Borough     object
Lat        float64
Long       float64
dtype: object

In [24]:
df2 = pd.read_csv("https://data.london.gov.uk/download/london-borough-profiles/c1693b82-68b1-44ee-beb2-3decf17dc1f8/london-borough-profiles.csv", encoding= 'unicode_escape')

In [25]:
df2.head()

,Code,Area_name,Inner/_Outer_London,GLA_Population_Estimate_2017,GLA_Household_Estimate_2017,Inland_Area_(Hectares),Population_density_(per_hectare)_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015",Net_internal_migration_(2015),Net_international_migration_(2015),Net_natural_change_(2015),%_of_resident_population_born_abroad_(2015),Largest_migrant_population_by_country_of_birth_(2011),%_of_largest_migrant_population_(2011),Second_largest_migrant_population_by_country_of_birth_(2011),%_of_second_largest_migrant_population_(2011),Third_largest_migrant_population_by_country_of_birth_(2011),%_of_third_largest_migrant_population_(2011),%_of_population_from_BAME_groups_(2016),%_people_aged_3+_whose_main_language_is_not_English_(2011_Census),"Overseas_nationals_entering_the_UK_(NINo),_(2015/16)","New_migrant_(NINo)_rates,_(2015/16)",Largest_migrant_population_arrived_during_2015/16,Second_largest_migrant_population_arrived_during_2015/16,Third_largest_migrant_population_arrived_during_2015/16,Employment_rate_(%)_(2015),Male_employment_rate_(2015),Female_employment_rate_(2015),Unemployment_rate_(2015),Youth_Unemployment_(claimant)_rate_18-24_(Dec-15),Proportion_of_16-18_year_olds_who_are_NEET_(%)_(2014),Proportion_of_the_working-age_population_who_claim_out-of-work_benefits_(%)_(May-2016),%_working-age_with_a_disability_(2015),Proportion_of_working_age_people_with_no_qualifications_(%)_2015,Proportion_of_working_age_with_degree_or_equivalent_and_above_(%)_2015,"Gross_Annual_Pay,_(2016)",Gross_Annual_Pay_-_Male_(2016),Gross_Annual_Pay_-_Female_(2016),Modelled_Household_median_income_estimates_2012/13,%_adults_that_volunteered_in_past_12_months_(2010/11_to_2012/13),Number_of_jobs_by_workplace_(2014),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),Crime_rates_per_thousand_population_2014/15,Fires_per_thousand_population_(2014),Ambulance_incidents_per_hundred_population_(2014),"Median_House_Price,_2015","Average_Band_D_Council_Tax_charge_(£),_2015/16",New_Homes_(net)_2015/16_(provisional),"Homes_Owned_outright,_(2014)_%","Being_bought_with_mortgage_or_loan,_(2014)_%","Rented_from_Local_Authority_or_Housing_Association,_(2014)_%","Rented_from_Private_landlord,_(2014)_%","%_of_area_that_is_Greenspace,_2005",Total_carbon_emissions_(2014),"Household_Waste_Recycling_Rate,_2014/15","Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)","%_of_adults_who_cycle_at_least_once_per_month,_2014/15","Average_Public_Transport_Accessibility_score,_2014","Achievement_of_5_or_more_A*-_C_grades_at_GCSE_or_equivalent_including_English_and_Maths,_2013/14",Rates_of_Children_Looked_After_(2016),%_of_pupils_whose_first_language_is_not_English_(2015),%_children_living_in_out-of-work_households_(2015),"Male_life_expectancy,_(2012-14)","Female_life_expectancy,_(2012-14)",Teenage_conception_rate_(2014),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),Childhood_Obesity_Prevalance_(%)_2015/16,People_aged_17+_with_diabetes_(%),Mortality_rate_from_causes_considered_preventable_2012/14,Political_control_in_council,Proportion_of_seats_won_by_Conservatives_in_2014_election,Proportion_of_seats_won_by_Labour_in_2014_election,Proportion_of_seats_won_by_Lib_Dems_in_2014_election,Turnout_at_2014_local_elections
0,E09000001,City of London,Inner London,8800,5326,290,30.3,43.2,11.4,73.1,15.5,-7,665,30,.,United States,2.8,France,2,Australia,1.9,27.5,17.1,975,152.2,India,France,United States,64.6,.,.,.,1.6,.,3.4,.,.,.,.,.,.,"£63,620",.,500400,3.4,84.3,26130,64.3,.,12.3,.,799999,931.2,80,.,.,.,.,4.8,1036,34.4,1692,0.4,16.9,7.9,78.6,101,.,7.9,.,.,.,6.6,7.1,6.0,5.6,NaN,2.6,129,.,.,.,.,.
1,E09000002,Barking and Dagenham,Outer London,209000,

In [26]:
df2 = df2.rename(columns={'Area_name':'Borough', 'Median_House_Price,_2015':'House_Price'})
df2.head()

,Code,Borough,Inner/_Outer_London,GLA_Population_Estimate_2017,GLA_Household_Estimate_2017,Inland_Area_(Hectares),Population_density_(per_hectare)_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015",Net_internal_migration_(2015),Net_international_migration_(2015),Net_natural_change_(2015),%_of_resident_population_born_abroad_(2015),Largest_migrant_population_by_country_of_birth_(2011),%_of_largest_migrant_population_(2011),Second_largest_migrant_population_by_country_of_birth_(2011),%_of_second_largest_migrant_population_(2011),Third_largest_migrant_population_by_country_of_birth_(2011),%_of_third_largest_migrant_population_(2011),%_of_population_from_BAME_groups_(2016),%_people_aged_3+_whose_main_language_is_not_English_(2011_Census),"Overseas_nationals_entering_the_UK_(NINo),_(2015/16)","New_migrant_(NINo)_rates,_(2015/16)",Largest_migrant_population_arrived_during_2015/16,Second_largest_migrant_population_arrived_during_2015/16,Third_largest_migrant_population_arrived_during_2015/16,Employment_rate_(%)_(2015),Male_employment_rate_(2015),Female_employment_rate_(2015),Unemployment_rate_(2015),Youth_Unemployment_(claimant)_rate_18-24_(Dec-15),Proportion_of_16-18_year_olds_who_are_NEET_(%)_(2014),Proportion_of_the_working-age_population_who_claim_out-of-work_benefits_(%)_(May-2016),%_working-age_with_a_disability_(2015),Proportion_of_working_age_people_with_no_qualifications_(%)_2015,Proportion_of_working_age_with_degree_or_equivalent_and_above_(%)_2015,"Gross_Annual_Pay,_(2016)",Gross_Annual_Pay_-_Male_(2016),Gross_Annual_Pay_-_Female_(2016),Modelled_Household_median_income_estimates_2012/13,%_adults_that_volunteered_in_past_12_months_(2010/11_to_2012/13),Number_of_jobs_by_workplace_(2014),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),Crime_rates_per_thousand_population_2014/15,Fires_per_thousand_population_(2014),Ambulance_incidents_per_hundred_population_(2014),House_Price,"Average_Band_D_Council_Tax_charge_(£),_2015/16",New_Homes_(net)_2015/16_(provisional),"Homes_Owned_outright,_(2014)_%","Being_bought_with_mortgage_or_loan,_(2014)_%","Rented_from_Local_Authority_or_Housing_Association,_(2014)_%","Rented_from_Private_landlord,_(2014)_%","%_of_area_that_is_Greenspace,_2005",Total_carbon_emissions_(2014),"Household_Waste_Recycling_Rate,_2014/15","Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)","%_of_adults_who_cycle_at_least_once_per_month,_2014/15","Average_Public_Transport_Accessibility_score,_2014","Achievement_of_5_or_more_A*-_C_grades_at_GCSE_or_equivalent_including_English_and_Maths,_2013/14",Rates_of_Children_Looked_After_(2016),%_of_pupils_whose_first_language_is_not_English_(2015),%_children_living_in_out-of-work_households_(2015),"Male_life_expectancy,_(2012-14)","Female_life_expectancy,_(2012-14)",Teenage_conception_rate_(2014),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),Childhood_Obesity_Prevalance_(%)_2015/16,People_aged_17+_with_diabetes_(%),Mortality_rate_from_causes_considered_preventable_2012/14,Political_control_in_council,Proportion_of_seats_won_by_Conservatives_in_2014_election,Proportion_of_seats_won_by_Labour_in_2014_election,Proportion_of_seats_won_by_Lib_Dems_in_2014_election,Turnout_at_2014_local_elections
0,E09000001,City of London,Inner London,8800,5326,290,30.3,43.2,11.4,73.1,15.5,-7,665,30,.,United States,2.8,France,2,Australia,1.9,27.5,17.1,975,152.2,India,France,United States,64.6,.,.,.,1.6,.,3.4,.,.,.,.,.,.,"£63,620",.,500400,3.4,84.3,26130,64.3,.,12.3,.,799999,931.2,80,.,.,.,.,4.8,1036,34.4,1692,0.4,16.9,7.9,78.6,101,.,7.9,.,.,.,6.6,7.1,6.0,5.6,NaN,2.6,129,.,.,.,.,.
1,E09000002,Barking and Dagenham,Outer London,209000,78188,"3,611",57.

In [27]:
df3 = df2[['Borough','House_Price']]
df3.head()

,Borough,House_Price
0,City of London,799999
1,Barking and Dagenham,243500
2,Barnet,445000
3,Bexley,275000
4,Brent,407250


In [28]:
df_londondata = pd.merge(df1, df3, on='Borough')
df_londondata

,Borough,Lat,Long,House_Price
0,City of London,51.5155,-0.0922,799999
1,Barking and Dagenham,51.5607,0.1557,243500
2,Barnet,51.6252,-0.1517,445000
3,Bexley,51.4549,0.1505,275000
4,Brent,51.5588,-0.2817,407250
5,Bromley,51.4039,0.0198,374975
6,Camden,51.5290,-0.1255,700000
7,Croydon,51.3714,-0.0977,300000
8,Ealing,51.5130,-0.3089,430000
9,Enfield,51.6538,-0.0799,320000


In [29]:
df_londondata.dtypes

Borough         object
Lat            float64
Long           float64
House_Price     object
dtype: object

In [30]:
df_londondata['House_Price']=df_londondata['House_Price'].astype(float)
df_londondata.dtypes

Borough         object
Lat            float64
Long           float64
House_Price    float64
dtype: object

In [31]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ldn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [158]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

df_londondata

# add markers to map
for lat, lng, label in zip(df_londondata['Lat'], df_londondata['Long'], df_londondata['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [32]:
CLIENT_ID = 'SU03R044TUVQOAVW14LYQDKJ2LYHHC0O2XAGZISBI3L1MCES' # your Foursquare ID
CLIENT_SECRET = 'OIYSPEJ5N140VXUCAFPIV51ULT0JOOTQZ3PCT12ODHNY4ZZW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SU03R044TUVQOAVW14LYQDKJ2LYHHC0O2XAGZISBI3L1MCES
CLIENT_SECRET:OIYSPEJ5N140VXUCAFPIV51ULT0JOOTQZ3PCT12ODHNY4ZZW


In [33]:
borough_lat = df_londondata.loc[0, 'Lat'] # neighborhood latitude value
borough_long = df_londondata.loc[0, 'Long'] # neighborhood longitude value

borough_name = df_londondata.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_lat, 
                                                               borough_long))

Latitude and longitude values of City of London are 51.5155, -0.0922.


In [34]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_lat, 
    borough_long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SU03R044TUVQOAVW14LYQDKJ2LYHHC0O2XAGZISBI3L1MCES&client_secret=OIYSPEJ5N140VXUCAFPIV51ULT0JOOTQZ3PCT12ODHNY4ZZW&v=20180605&ll=51.5155,-0.0922&radius=500&limit=100'

In [35]:
results = requests.get(url).json()

In [36]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Goodman Steak House Restaurant,Steakhouse,51.514398,-0.090745
1,The Ned Hotel,Hotel,51.513755,-0.090067
2,The Merchant House,Cocktail Bar,51.513264,-0.093039
3,Hawksmoor Guildhall,Steakhouse,51.515498,-0.090849
4,City Càphê,Vietnamese Restaurant,51.514750,-0.091545


In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


100 venues were returned by Foursquare.


In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
london_venues = getNearbyVenues(names=df_londondata['Borough'],
                                   latitudes=df_londondata['Lat'],
                                   longitudes=df_londondata['Long']
                                  )

City of London
Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


In [47]:
print(london_venues.shape)
london_venues.head()

(1528, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.5155,-0.0922,Goodman Steak House Restaurant,51.514398,-0.090745,Steakhouse
1,City of London,51.5155,-0.0922,The Ned Hotel,51.513755,-0.090067,Hotel
2,City of London,51.5155,-0.0922,The Merchant House,51.513264,-0.093039,Cocktail Bar
3,City of London,51.5155,-0.0922,Hawksmoor Guildhall,51.515498,-0.090849,Steakhouse
4,City of London,51.5155,-0.0922,City Càphê,51.514750,-0.091545,Vietnamese Restaurant


In [49]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 221 uniques categories.


In [50]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Borough'] = london_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Borough,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chaat Place,Champagne Bar,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoothie Shop,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [51]:
london_onehot.shape

(1528, 222)

In [52]:
london_grouped = london_onehot.groupby('Borough').mean().reset_index()
london_grouped

,Borough,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chaat Place,Champagne Bar,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoothie Shop,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [53]:
london_grouped.shape


(33, 222)

In [54]:
num_top_venues = 10

for hood in london_grouped['Borough']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
                    venue  freq
0    Gym / Fitness Center  0.14
1                    Pool  0.14
2             Bus Station  0.14
3             Supermarket  0.14
4             Golf Course  0.14
5                    Park  0.14
6       Martial Arts Dojo  0.14
7      African Restaurant  0.00
8            Noodle House  0.00
9  Okonomiyaki Restaurant  0.00


----Barnet----
                    venue  freq
0                    Café  0.67
1                Bus Stop  0.33
2      African Restaurant  0.00
3                 Rafting  0.00
4            Noodle House  0.00
5  Okonomiyaki Restaurant  0.00
6            Optical Shop  0.00
7         Organic Grocery  0.00
8     Outdoor Event Space  0.00
9       Outdoor Sculpture  0.00


----Bexley----
                  venue  freq
0        Clothing Store  0.10
1           Coffee Shop  0.10
2                   Pub  0.10
3           Supermarket  0.07
4    Italian Restaurant  0.07
5  Fast Food Restaurant  0.07
6              Pharmacy

                   venue  freq
0         Clothing Store  0.11
1   Fast Food Restaurant  0.08
2            Supermarket  0.08
3          Grocery Store  0.05
4                 Bakery  0.05
5       Department Store  0.05
6            Coffee Shop  0.05
7         Sandwich Place  0.05
8  Portuguese Restaurant  0.03
9          Movie Theater  0.03


----Richmond upon Thames----
                venue  freq
0                 Pub  0.16
1         Coffee Shop  0.12
2  Italian Restaurant  0.08
3                Café  0.06
4       Grocery Store  0.04
5            Bus Stop  0.04
6            Pharmacy  0.04
7   Indian Restaurant  0.04
8    Stationery Store  0.02
9          Steakhouse  0.02


----Southwark----
                venue  freq
0         Coffee Shop  0.10
1                 Pub  0.06
2                 Bar  0.05
3   French Restaurant  0.04
4        Cocktail Bar  0.04
5      Scenic Lookout  0.04
6                Café  0.03
7         Pizza Place  0.03
8  Italian Restaurant  0.03
9          Restauran

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = london_grouped['Borough']

for ind in np.arange(london_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Pool,Gym / Fitness Center,Bus Station,Supermarket,Martial Arts Dojo,Park,Golf Course,Department Store,Electronics Store,Film Studio
1,Barnet,Café,Bus Stop,Yoga Studio,Creperie,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market
2,Bexley,Pub,Coffee Shop,Clothing Store,Fast Food Restaurant,Pharmacy,Supermarket,Italian Restaurant,Shopping Mall,Sandwich Place,Furniture / Home Store
3,Brent,Coffee Shop,Hotel,Clothing Store,Bar,Sporting Goods Shop,Grocery Store,Italian Restaurant,Indian Restaurant,Sandwich Place,Burger Joint
4,Bromley,Clothing Store,Coffee Shop,Burger Joint,Pizza Place,Bar,Café,Sandwich Place,English Restaurant,Electronics Store,Furniture / Home Store


### Cluster Neighbourhoods. 

In [57]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [58]:
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [59]:
london_merged = df_londondata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')

london_merged.head() # check the last columns!

,Borough,Lat,Long,House_Price,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,City of London,51.5155,-0.0922,799999.0,1,Coffee Shop,Italian Restaurant,Hotel,French Restaurant,Gym / Fitness Center,Clothing Store,Café,Steakhouse,Scenic Lookout,Wine Bar
1,Barking and Dagenham,51.5607,0.1557,243500.0,0,Pool,Gym / Fitness Center,Bus Station,Supermarket,Martial Arts Dojo,Park,Golf Course,Department Store,Electronics Store,Film Studio
2,Barnet,51.6252,-0.1517,445000.0,2,Café,Bus Stop,Yoga Studio,Creperie,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market
3,Bexley,51.4549,0.1505,275000.0,1,Pub,Coffee Shop,Clothing Store,Fast Food Restaurant,Pharmacy,Supermarket,Italian Restaurant,Shopping Mall,Sandwich Place,Furniture / Home Store
4,Brent,51.5588,-0.2817,407250.0,1,Coffee Shop,Hotel,Clothing Store,Bar,Sporting Goods Shop,Grocery Store,Italian Restaurant,Indian Restaurant,Sandwich Place,Burger Joint


In [60]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Lat'], london_merged['Long'], london_merged['Borough'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [64]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Barking and Dagenham,Pool,Gym / Fitness Center,Bus Station,Supermarket,Martial Arts Dojo,Park,Golf Course,Department Store,Electronics Store,Film Studio
13,Haringey,Fast Food Restaurant,Hotel Bar,Gym / Fitness Center,Bus Stop,Café,Mediterranean Restaurant,Supermarket,Light Rail Station,Bar,Bakery
14,Harrow,Indie Movie Theater,Supermarket,Grocery Store,Platform,Coffee Shop,Indian Restaurant,Antique Shop,Fried Chicken Joint,Food Court,Flea Market
22,Lewisham,Supermarket,Grocery Store,Platform,Bus Stop,Coffee Shop,Italian Restaurant,Train Station,Pub,Fast Food Restaurant,Dessert Shop
23,Merton,Café,Supermarket,Italian Restaurant,Park,Sandwich Place,Garden Center,Burger Joint,Bar,Bakery,Grocery Store


#### Cluster 2

In [65]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,City of London,Coffee Shop,Italian Restaurant,Hotel,French Restaurant,Gym / Fitness Center,Clothing Store,Café,Steakhouse,Scenic Lookout,Wine Bar
3,Bexley,Pub,Coffee Shop,Clothing Store,Fast Food Restaurant,Pharmacy,Supermarket,Italian Restaurant,Shopping Mall,Sandwich Place,Furniture / Home Store
4,Brent,Coffee Shop,Hotel,Clothing Store,Bar,Sporting Goods Shop,Grocery Store,Italian Restaurant,Indian Restaurant,Sandwich Place,Burger Joint
5,Bromley,Clothing Store,Coffee Shop,Burger Joint,Pizza Place,Bar,Café,Sandwich Place,English Restaurant,Electronics Store,Furniture / Home Store
6,Camden,Café,Coffee Shop,Hotel,Pub,Burger Joint,Garden,Bar,Breakfast Spot,Train Station,Pizza Place
7,Croydon,Coffee Shop,Pub,Portuguese Restaurant,Brewery,Burger Joint,Strip Club,Gaming Cafe,Bookstore,Breakfast Spot,Spanish Restaurant
8,Ealing,Coffee Shop,Pub,Clothing Store,Park,Bakery,Burger Joint,Bus Stop,Café,Pizza Place,Italian Restaurant
9,Enfield,Clothing Store,Coffee Shop,Pub,Café,Optical Shop,Supermarket,Shopping Mall,Department Store,Video Game Store,Pharmacy
10,Greenwich,Pub,Fast Food Restaurant,Coffee Shop,Clothing Store,Supermarket,Pharmacy,Plaza,Hotel,Grocery Store,Steakhouse
11,Hackney,Café,Pub,Brewery,Coffee Shop,Bakery,Grocery Store,Modern European Restaurant,Clothing Store,Cocktail Bar,Vegetarian / Vegan Restaurant


#### Cluster 3

In [66]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Barnet,Café,Bus Stop,Yoga Studio,Creperie,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market


#### Cluster 4

In [69]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Hounslow,Bed & Breakfast,Chinese Restaurant,Café,Park,Yoga Studio,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant


#### Cluster 5

In [73]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]


,Lat,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,51.5077,Hotel,Airport Service,Currency Exchange,Duty-free Shop,Sandwich Place,Pharmacy,Rafting,Chinese Restaurant,Airport Lounge,Airport


In [ ]:
df4 = pd.read_csv("https://data.london.gov.uk/download/london-borough-profiles/c1693b82-68b1-44ee-beb2-3decf17dc1f8/london-borough-profiles.csv", encoding= 'unicode_escape')